In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../train.csv')
print(df.shape)
df.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [57]:
month_ini = '2011-07'
month_end = '2011-08'

In [58]:
# df_train = df.copy()
# df_train['month'] = pd.DatetimeIndex(df_train['datetime']).month
# df_train = df[df['datetime'] < '2011-02'].copy()
df_train = df[(df['datetime'] > month_ini) & (df['datetime'] < month_end)].copy()
df_train['day'] = pd.DatetimeIndex(df_train['datetime']).day
df_train['dayofweek'] = pd.DatetimeIndex(df_train['datetime']).dayofweek
df_train['hour'] = pd.DatetimeIndex(df_train['datetime']).hour
df_train = df_train.drop('datetime', axis=1)
y = df_train['count'].copy()
df_train = df_train.drop(['casual','registered','count'], axis=1)
# df = df.drop(['casual','registered'], axis=1)
print(df_train.shape)
df_train.head()

(456, 11)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,day,dayofweek,hour
2690,3,0,1,1,27.06,31.060,50,0.0,1,4,0
2691,3,0,1,1,27.06,31.060,50,0.0,1,4,1
2692,3,0,1,1,25.42,30.305,69,0.0,1,4,2
2693,3,0,1,1,26.24,31.060,53,0.0,1,4,3
2694,3,0,1,1,25.42,31.060,57,0.0,1,4,4


In [27]:
# define base mode
def baseline_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(n_features, input_dim=n_features, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [7]:
# estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, nb_epoch=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

In [9]:
X = df_train.values
Y = y.values
kfold = KFold(n_splits=10)
# %time results = cross_val_score(estimator, X, Y, cv=kfold, verbose=2)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[CV]  ................................................................
[CV] ................................................. , total=  10.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.1s remaining:    0.0s


[CV] ................................................. , total=  10.3s
[CV]  ................................................................
[CV] ................................................. , total=  10.7s
[CV]  ................................................................
[CV] ................................................. , total=  10.9s
[CV]  ................................................................
[CV] ................................................. , total=  10.9s
[CV]  ................................................................
[CV] ................................................. , total=  11.3s
[CV]  ................................................................
[CV] ................................................. , total=  11.3s
[CV]  ................................................................
[CV] ................................................. , total=  11.6s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.9min finished


In [28]:
def larger_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(n_features, input_dim=n_features, init='normal', activation='relu'))
    model.add(Dense(6, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [29]:
X = df_train.values
Y = y.values
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, nb_epoch=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

CPU times: user 5min 4s, sys: 17.4 s, total: 5min 21s
Wall time: 3min 36s
Larger: 1818.82 (703.53) MSE


In [4]:
def wider_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=n_features, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [5]:
X = df_train.values
Y = y.values
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, nb_epoch=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[CV]  ................................................................
[CV] ................................................. , total=   8.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s remaining:    0.0s


[CV] ................................................. , total=   8.2s
[CV]  ................................................................
[CV] ................................................. , total=   8.4s
[CV]  ................................................................
[CV] ................................................. , total=   8.3s
[CV]  ................................................................
[CV] ................................................. , total=   8.7s
[CV]  ................................................................
[CV] ................................................. , total=   8.8s
[CV]  ................................................................
[CV] ................................................. , total=   9.1s
[CV]  ................................................................
[CV] ................................................. , total=   9.3s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.5min finished


In [8]:
def new_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=n_features, init='normal', activation='relu'))
#     model.add(Dense(6, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [10]:
X = df_train.values
Y = y.values
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[CV]  ................................................................
[CV] ................................................. , total=  22.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.2s remaining:    0.0s


[CV] ................................................. , total=  22.2s
[CV]  ................................................................
[CV] ................................................. , total=  22.9s
[CV]  ................................................................
[CV] ................................................. , total=  23.1s
[CV]  ................................................................
[CV] ................................................. , total=  23.8s
[CV]  ................................................................
[CV] ................................................. , total=  23.7s
[CV]  ................................................................
[CV] ................................................. , total=  23.7s
[CV]  ................................................................
[CV] ................................................. , total=  24.6s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.9min finished


In [12]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=7)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[CV]  ................................................................
[CV] ................................................. , total=  25.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   25.2s remaining:    0.0s


[CV] ................................................. , total=  25.7s
[CV]  ................................................................
[CV] ................................................. , total=  25.8s
[CV]  ................................................................
[CV] ................................................. , total=  25.8s
[CV]  ................................................................
[CV] ................................................. , total=  25.9s
[CV]  ................................................................
[CV] ................................................. , total=  26.0s
[CV]  ................................................................
[CV] ................................................. , total=  26.3s
CPU times: user 5min 1s, sys: 25.1 s, total: 5min 26s
Wall time: 3min
Results: 1680.77 (499.13) MSE


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  3.0min finished


In [16]:
def new_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(50, input_dim=n_features, init='normal', activation='relu'))
#     model.add(Dense(4, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=7)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[CV]  ................................................................
[CV] ................................................. , total=  33.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   33.2s remaining:    0.0s


[CV] ................................................. , total=  33.9s
[CV]  ................................................................
[CV] ................................................. , total=  33.4s
[CV]  ................................................................
[CV] ................................................. , total=  34.0s
[CV]  ................................................................
[CV] ................................................. , total=  34.3s
[CV]  ................................................................
[CV] ................................................. , total=  36.3s
[CV]  ................................................................
[CV] ................................................. , total=  37.4s
CPU times: user 6min 1s, sys: 24.9 s, total: 6min 26s
Wall time: 4min 2s
Results: 1674.07 (474.08) MSE


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  4.0min finished


In [4]:
X = df_train.values
Y = y.values

In [21]:
def new_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(50, input_dim=n_features, init='normal', activation='relu'))
#     model.add(Dense(4, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=7)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[CV]  ................................................................
[CV] ................................................. , total= 6.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.4min remaining:    0.0s


[CV] ................................................. , total= 6.4min
[CV]  ................................................................
[CV] ................................................. , total= 6.6min
[CV]  ................................................................
[CV] ................................................. , total= 6.6min
[CV]  ................................................................
[CV] ................................................. , total= 6.2min
[CV]  ................................................................
[CV] ................................................. , total= 6.2min
[CV]  ................................................................
[CV] ................................................. , total= 6.4min
CPU times: user 1h 35min 9s, sys: 10min 25s, total: 1h 45min 34s
Wall time: 44min 46s
Results: 19744.47 (8402.78) MSE


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed: 44.8min finished


In [59]:
X = df_train.values
Y = y.values
def new_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(50, input_dim=n_features, init='normal', activation='relu'))
#     model.add(Dense(4, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
estimators = []
estimators.append(('scale', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
pipeline.fit(X, Y)
kfold = KFold(n_splits=7)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[CV]  ................................................................
[CV] ................................................. , total=  23.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.6s remaining:    0.0s


[CV] ................................................. , total=  23.9s
[CV]  ................................................................
[CV] ................................................. , total=  23.2s
[CV]  ................................................................
[CV] ................................................. , total=  22.5s
[CV]  ................................................................
[CV] ................................................. , total=  25.0s
[CV]  ................................................................
[CV] ................................................. , total=  24.6s
[CV]  ................................................................
[CV] ................................................. , total=  24.8s
CPU times: user 3min 21s, sys: 13.1 s, total: 3min 34s
Wall time: 2min 47s
Results: 16057.80 (6740.01) MSE


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  2.8min finished


## Prediction

In [7]:
df_test = pd.read_csv('../test.csv')
df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [61]:
# dft = df_test.copy()
# dft['month'] = pd.DatetimeIndex(dft['datetime']).month
# dft = df_test[df_test['datetime'] < '2011-02'].copy()
dft = df_test[(df_test['datetime'] > month_ini) & (df_test['datetime'] < month_end)].copy()
dft['day'] = pd.DatetimeIndex(dft['datetime']).day
dft['dayofweek'] = pd.DatetimeIndex(dft['datetime']).dayofweek
dft['hour'] = pd.DatetimeIndex(dft['datetime']).hour
dft = dft.drop('datetime', axis=1)
print(dft.shape)
dft.head()

(288, 11)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,day,dayofweek,hour
1560,3,0,1,1,28.70,33.335,84,7.0015,20,2,0
1561,3,0,1,1,28.70,33.335,84,0.0000,20,2,1
1562,3,0,1,1,28.70,33.335,84,0.0000,20,2,2
1563,3,0,1,1,28.70,33.335,84,7.0015,20,2,3
1564,3,0,1,1,27.88,31.820,83,0.0000,20,2,4


In [62]:
test_pred = pipeline.predict(dft)
test_pred[test_pred < 0] = 0

# str_prediction = "datetime,count\n"
for i in range(0, len(test_pred)):
    datetime = df_test['datetime'][i]
    pred = int(round(test_pred[i]))
    str_prediction += "{},{}\n".format(datetime, pred)

# print(str_prediction)
f = open('pred_keras.csv','a')
f.write(str_prediction)
f.close()

In [60]:
results

array([  7649.53538374,  28948.26125629,  16127.75179349,   8268.95859469,
        20048.92442909,  17137.67294546,  14223.50642278])